In [1]:
import os, sys, glob, json, scipy as sp
import matplotlib.pyplot as plt
os.chdir("../")
path = os.getcwd()
sys.path.insert(0, path)
import GWGen
from GWGen.WFGenerator import *
from GWGen.DressedFluxes.Fluxes import *

In [2]:
pc = ProcaSolution(1e6,0.9,1e-17)
pn = PN(1e6,1)

UltralightBoson instance not provided. Instantiating class...
done.


In [23]:
etest = 0.8
ptest = 10
M=1e6*unit.M_sun
m=1*unit.M_sun

print("\tDimensionful fluxes\n")
print("E: ", pn.UndressedEFlux(etest, ptest)*pn.epsilon*(cons.c**5/cons.G).decompose().value)
print(r"E(proca): ",pc.ChangeInOrbitalConstants()["E"](0,etest,ptest))
print("L: ", pn.UndressedLFlux(etest, ptest)*(pn.epsilon*M*cons.c**2).decompose().value)
print(r"L(proca): ",pc.ChangeInOrbitalConstants()["L"](0,etest,ptest))


print("\n\tAdimensionalize proca flux\n")
print("E: ", pn.UndressedEFlux(etest, ptest))
print("E(proca): ", pc.ChangeInOrbitalConstants()["E"](0,etest,ptest)*(unit.kg*(unit.m**2/(unit.s**3)))/pn.epsilon * (cons.G/(cons.c**5)).decompose())
print("L: ", pn.UndressedLFlux(etest, ptest))
print("L(proca): ", pc.ChangeInOrbitalConstants()["L"](0,etest,ptest)*(unit.kg*(unit.m/unit.s)**2)/(pn.epsilon*M*cons.c**2).decompose())


	Dimensionful fluxes

E:  -1.617097043827651e+36
E(proca):  2.6287260438406194e+36
L:  -1.2754000352719981e+38
L(proca):  1.8139508939042193e+39

	Adimensionalize proca flux

E:  -4.456955441200019e-11
E(proca):  7.245152595658534e-11
L:  -7.13672740036236e-10
L(proca):  1.015028437307294e-08


In [22]:
unit.kg*cons.G/cons.c**2

<Quantity 7.42616027e-28 m>